## About this code
The purpose of this notebook is to show an example of how we can extract information from web pages with some level of accuracy.
I've made this notebook as an example to help you in case you needed to scrape infromation in the future.
Here, we are collecting the speeches of previous presidents of the united states

In [ ]:
import os
from os import walk
from bs4 import BeautifulSoup
import requests

### Files structures
The folders exist each named by the name of some of the american presidents
inside each folder, there is an html file "initial.txt" scraped from millercenter.org
containing links for all the speeches of each corresponding president

In [ ]:
# parent folder
path = 'presidents-speeches/'

# will contain all the data "president name" : ["speeches links"]
presidential_dictionary_list = []
for (root, dirs, files) in walk(path):
    # go through child folders, skip the first parent folder
    if len(dirs) == 1:
        president_speahces_dict = {}
        # get the name of the president from the file path
        president_name = root.split("/")[1]
        president_speahces_dict['president_name'] = president_name
        speches_links = []
        file_path = "{}/{}".format(root,files[0])
        # open the "initial.txt" file for each president
        with open(file_path) as html_links_file:
            soup = BeautifulSoup(html_links_file)
            for link in soup.findAll('a'):
                # extract the speeches links from the <a> tags
                speches_links.append(link.get('href'))
        president_speahces_dict['speeches_links'] = speches_links
        presidential_dictionary_list.append(president_speahces_dict)

In [ ]:
presidential_dictionary_list

[{'president_name': 'Donald Trump',
  'speeches_links': ['/the-presidency/presidential-speeches/january-8-2020-statement-iran',
   '/the-presidency/presidential-speeches/january-3-2020-remarks-killing-qasem-soleimani',
   '/the-presidency/presidential-speeches/october-27-2019-statement-death-abu-bakr-al-baghdadi',
   '/the-presidency/presidential-speeches/september-25-2019-press-conference',
   '/the-presidency/presidential-speeches/september-24-2019-remarks-united-nations-general-assembly',
   '/the-presidency/presidential-speeches/february-15-2019-speech-declaring-national-emergency',
   '/the-presidency/presidential-speeches/february-5-2019-state-union-address',
   '/the-presidency/presidential-speeches/january-19-2019-remarks-about-us-southern-border',
   '/the-presidency/presidential-speeches/september-25-2018-address-73rd-session-united-nations-general',
   '/the-presidency/presidential-speeches/july-24-2018-speech-veterans-foreign-wars-national-convention',
   '/the-presidency/p

### Fetch speeches from the site
now that we have the links of the speeches, we can fetch them and extract the content
we will save them in files titled by date__title, corresponding to every president

In [ ]:
# format speech titles to be appropriate for usage as file names
import re
def file_name_format(text):
    return re.sub('[^A-Za-z0-9\-\_]+', '', text)

In [ ]:
for presidential_dictionary in presidential_dictionary_list:
    print("extracting speeches of president {}".format(presidential_dictionary['president_name']))
    speech_counter = 1
    total_speeches = len(presidential_dictionary['speeches_links'])
    for speech_link in presidential_dictionary['speeches_links']:
        try:
            resp = requests.get("https://millercenter.org/{}".format(speech_link))
        except requests.exceptions.RequestException as e:
            print(e)
        print("speech {}/{}".format(speech_counter,total_speeches))
        speech_counter+=1
        if resp.status_code == 200:
            soup = BeautifulSoup(resp._content)
            president_speech = ''
            # the speech is present inside a <div> with class "view-transcript", as a set of <p> tags
            for transcripts in soup.find_all('div', { "class" : "view-transcript"}): # there is only one view-transcript per page
                ps = transcripts.find_all('p')
                president_speech = ' '.join([p.text for p in ps])
            # get the title and date of every speech, and name the file accordingly
            for title in soup.find_all('h2', { "class" : "presidential-speeches--title"}):
                ttl = title.find('span').text
                date_speech_title = ttl.split(":")
                date = date_speech_title[0].strip()
                speech_title = date_speech_title[1].strip()
            with open(path+presidential_dictionary['president_name']+"/speeches/"+date+"__"+file_name_format(speech_title)+".txt","w+") as out_file:
                out_file.write(str(president_speech.encode('utf8')))

extracting speeches of president Donald Trump
speech 1/22
speech 2/22
speech 3/22
speech 4/22
speech 5/22
speech 6/22
speech 7/22
speech 8/22
speech 9/22
speech 10/22
speech 11/22
speech 12/22
speech 13/22
speech 14/22
speech 15/22
speech 16/22
speech 17/22
speech 18/22
speech 19/22
speech 20/22
speech 21/22
speech 22/22
extracting speeches of president Richard M. Nixon
speech 1/23
speech 2/23
speech 3/23
speech 4/23
speech 5/23
speech 6/23
speech 7/23
speech 8/23
speech 9/23
speech 10/23
speech 11/23
speech 12/23
speech 13/23
speech 14/23
speech 15/23
speech 16/23
speech 17/23
speech 18/23
speech 19/23
speech 20/23
speech 21/23
speech 22/23
speech 23/23
extracting speeches of president Andrew Jackson
speech 1/26
speech 2/26
speech 3/26
speech 4/26
speech 5/26
speech 6/26
speech 7/26
speech 8/26
speech 9/26
speech 10/26
speech 11/26
speech 12/26
speech 13/26
speech 14/26
speech 15/26
speech 16/26
speech 17/26
speech 18/26
speech 19/26
speech 20/26
speech 21/26
speech 22/26
speech 23/26

### fin
After running this code, all the speeches are be saved inside their corresponding president folder

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d42f6cca-4e71-466f-a1d5-45dcc7619b1d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>